In [1]:
import os
os.getcwd()
os.chdir('/home/siyuan/Seafile/Thesis/PycharmProjects/DD_PE_paper/')

In [2]:
from qiskit.transpiler import PassManager, InstructionDurations
from src.tools.DD_insertion import construct_bv_circuit, \
                                 construct_graph_matrix, \
                                 convert_count_to_prob, \
                                 translate_circuit_to_basis
from qiskit.circuit.library import QFT, GraphState

In [3]:
from qiskit import IBMQ, transpile, Aer
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-france', group='univ-montpellier', project='default')
backend = provider.get_backend('ibmq_toronto')

In [4]:
bv_circuits = []
hs_circuits = []
qft_circuits = []
for i in range(3, 15):
    bv_circuits.append(construct_bv_circuit(i))

# for i in range(2, 15, 2):
#     hs_circuits.append(construct_hs_circuit(i))

for i in range(3, 15):
    qft_circuits.append(QFT(i))

for circuit in bv_circuits:
    circuit.measure_all()

# for circuit in hs_circuits:
#     circuit.measure_all()

for circuit in qft_circuits:
    circuit.measure_all()

In [5]:
durations = InstructionDurations.from_backend(backend)
## add duration of y gates which are used for DD sequences
bconf = backend.configuration()
for i in range(bconf.num_qubits):
    x_duration = durations.get('x', i)
    durations.update(InstructionDurations(
        [('y', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('rx', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('ry', i, x_duration)]
        ))

In [6]:
graph_state_circuits = []
coupling_map = backend.configuration().coupling_map

for i in range(3, 15):
    gs_circuit_matrix = construct_graph_matrix(i, coupling_map)
    graph_state_circuits.append(GraphState(gs_circuit_matrix))

for circuit in graph_state_circuits:
    circuit.measure_all()

In [7]:
from src.tools.DD_insertion import pm_DD_sequences
pms = pm_DD_sequences(durations)

In [8]:
bv_job_ids = []
bv_jobs = []

for circuit in bv_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    bv_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    bv_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
6295d6cb4ad6558e869e4aaa
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
6295d6d89baeab33384acd7a
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d6da3bb4afcd82b9550e
if applied DD for None: False
if applied DD fo

In [9]:
for job_id in bv_job_ids:
    print(job_id)

6295d6cb4ad6558e869e4aaa
6295d6d89baeab33384acd7a
6295d6da3bb4afcd82b9550e
6295d6dd0212fffb86251f20
6295d6e09baeaba2e84acd7c
6295d6e3f33cdb23922e06a5
6295d6ee1ea57f4bd7ebd626
6295d6fe3bb4afc3dcb9550f
6295d7014ad655645f9e4aac
6295d70ef33cdb36ec2e06a8
6295d71916cdc4490ef25e4b
6295d71df33cdb20122e06aa


In [7]:
from qiskit import execute

shots = 8192
psts = []
bv_job_ids = [
    '6295d6cb4ad6558e869e4aaa',
    '6295d6d89baeab33384acd7a',
    '6295d6da3bb4afcd82b9550e',
    '6295d6dd0212fffb86251f20',
    '6295d6e09baeaba2e84acd7c',
    '6295d6e3f33cdb23922e06a5',
    '6295d6ee1ea57f4bd7ebd626',
    '6295d6fe3bb4afc3dcb9550f',
    '6295d7014ad655645f9e4aac',
    '6295d70ef33cdb36ec2e06a8',
    '6295d71916cdc4490ef25e4b',
    '6295d71df33cdb20122e06aa',

]
for i, circuit in enumerate(bv_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    ideal_result = list(ideal_counts.keys())[0]
    result_counts = backend.retrieve_job(bv_job_ids[i]).result().get_counts()
    pst = []
    for result_count in result_counts:
        obtain_results = result_count.get(ideal_result, 0)
        pst.append(obtain_results/shots)
    psts.append(pst)

bv_circuit_names = ['BV3', 'BV4', 'BV5', 'BV6', 'BV7', 'BV8', 'BV9', 'BV10', 'BV11', 'BV12', 'BV13', 'BV14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']
bv_results_data = {}

for i, name in enumerate(bv_circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, pst in enumerate(psts[i]):
        print(f'pst result of {DD_sequences[j]}: {pst}')
    bv_results_data[name] = psts[i]

---------Result of circuit BV3 -------------
pst result of no_dd: 0.4735107421875
pst result of hahn_X: 0.4794921875
pst result of hahn_Y: 0.4888916015625
pst result of CP: 0.48193359375
pst result of CPMG: 0.491455078125
pst result of xy4: 0.4859619140625
pst result of xy8: 0.499267578125
pst result of xy16: 0.4808349609375
pst result of udd1: 0.484130859375
pst result of udd2: 0.4613037109375
pst result of kdd: 0.49169921875
---------Result of circuit BV4 -------------
pst result of no_dd: 0.483154296875
pst result of hahn_X: 0.477783203125
pst result of hahn_Y: 0.480224609375
pst result of CP: 0.48828125
pst result of CPMG: 0.4398193359375
pst result of xy4: 0.44580078125
pst result of xy8: 0.4617919921875
pst result of xy16: 0.4307861328125
pst result of udd1: 0.4859619140625
pst result of udd2: 0.4290771484375
pst result of kdd: 0.475830078125
---------Result of circuit BV5 -------------
pst result of no_dd: 0.2232666015625
pst result of hahn_X: 0.231689453125
pst result of hahn_Y

In [8]:
bv_results_data

{'BV3': [0.4735107421875,
  0.4794921875,
  0.4888916015625,
  0.48193359375,
  0.491455078125,
  0.4859619140625,
  0.499267578125,
  0.4808349609375,
  0.484130859375,
  0.4613037109375,
  0.49169921875],
 'BV4': [0.483154296875,
  0.477783203125,
  0.480224609375,
  0.48828125,
  0.4398193359375,
  0.44580078125,
  0.4617919921875,
  0.4307861328125,
  0.4859619140625,
  0.4290771484375,
  0.475830078125],
 'BV5': [0.2232666015625,
  0.231689453125,
  0.2274169921875,
  0.245361328125,
  0.2327880859375,
  0.239990234375,
  0.256591796875,
  0.2215576171875,
  0.2333984375,
  0.2454833984375,
  0.228271484375],
 'BV6': [0.1475830078125,
  0.1446533203125,
  0.147216796875,
  0.2603759765625,
  0.248779296875,
  0.25244140625,
  0.16015625,
  0.2222900390625,
  0.163818359375,
  0.1810302734375,
  0.159423828125],
 'BV7': [0.0150146484375,
  0.015869140625,
  0.0140380859375,
  0.1072998046875,
  0.099853515625,
  0.1165771484375,
  0.14990234375,
  0.1025390625,
  0.1561279296875,
 

In [9]:
# import numpy as np
# import matplotlib.pyplot as plt

# baseline = []
# data = []
# for name in bv_circuit_names:
#     result_data = bv_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [i / result_data[0] for i in result_data[1:]]
#     data.append(ratio)

# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[1:, 2]
# cpmg = data[1:, 3]
# xy4 = data[1:, 4]
# xy8 = data[1:, 5]
# xy16 = list(data[2:, 6])
# xy16.insert(0, None)
# udd_X = data[1:, 7]
# udd_Y = data[1:, 8]
# kdd = list(data[2:, 9])
# kdd.insert(0, None)

# circuit_names = ['BV3', 'BV4', 'BV5', 'BV6', 'BV7', 'BV8', 'BV9', 'BV10', 'BV11', 'BV12', ]

# x_ticks = circuit_names[:6]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure

# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0, 0, 1, 1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2][:6], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2][:6], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2][:6], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2][:6], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2][:6], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2][:6], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2][:6], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2][:6], linestyle='--', marker='*', color='m', label='kdd')

# plt.axhline(1, color='tab:brown', lw=2)

# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative PST result for BV circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative PST', fontsize=16)
# plt.savefig('TQE_Relat_PST_BV_toronto_0530.pdf', bbox_inches='tight', pad_inches=0)

In [10]:
gs_job_ids = []
gs_jobs = []

for circuit in graph_state_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    gs_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    gs_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d71f1ea57f3e92ebd628
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d72016cdc49eecf25e4c
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d7224bb9754326efb4d3
if applied DD for None: False
if applied DD for N

In [11]:
for job_id in gs_job_ids:
    print(job_id)

6295d71f1ea57f3e92ebd628
6295d72016cdc49eecf25e4c
6295d7224bb9754326efb4d3
6295d7241ea57f8d42ebd629
6295d72516cdc45eeff25e4d
6295d72732e176b25993c9e7
6295d7290212ffdd89251f22
6295d72b16cdc42e10f25e4e
6295d72d32e17659b793c9e8
6295d72ff33cdb5a182e06ab
6295d7314ad65585a59e4aae
6295d7339baeab77344acd7e


In [10]:
from scipy.spatial.distance import jensenshannon
from qiskit import execute
gs_jsds = []

gs_job_ids = [
    '6295d71f1ea57f3e92ebd628',
    '6295d72016cdc49eecf25e4c',
    '6295d7224bb9754326efb4d3',
    '6295d7241ea57f8d42ebd629',
    '6295d72516cdc45eeff25e4d',
    '6295d72732e176b25993c9e7',
    '6295d7290212ffdd89251f22',
    '6295d72b16cdc42e10f25e4e',
    '6295d72d32e17659b793c9e8',
    '6295d72ff33cdb5a182e06ab',
    '6295d7314ad65585a59e4aae',
    '6295d7339baeab77344acd7e',
]


shots = 8192

for i, circuit in enumerate(graph_state_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(gs_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    gs_jsds.append(jsd)

circuit_names = ['GS3', 'GS4', 'GS5', 'GS6', 'GS7', 'GS8', 'GS9', 'GS10', 'GS11', 'GS12', 'GS13', 'GS14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

gs_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(gs_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    gs_results_data[name] = gs_jsds[i]

---------Result of circuit GS3 -------------
jsd result of no_dd: 0.03900207408294435
jsd result of hahn_X: 0.03182899355288701
jsd result of hahn_Y: 0.04340882872787646
jsd result of CP: 0.06359335351421432
jsd result of CPMG: 0.03202563169941431
jsd result of xy4: 0.0679296545368999
jsd result of xy8: 0.05139029924730704
jsd result of xy16: 0.04432338990842704
jsd result of udd1: 0.11887832300836947
jsd result of udd2: 0.03970590428297073
jsd result of kdd: 0.03105083108013777
---------Result of circuit GS4 -------------
jsd result of no_dd: 0.035637905844616076
jsd result of hahn_X: 0.04310968907058414
jsd result of hahn_Y: 0.039363888799197
jsd result of CP: 0.04415232832299162
jsd result of CPMG: 0.036547246485533716
jsd result of xy4: 0.04606141538586049
jsd result of xy8: 0.059055680890087794
jsd result of xy16: 0.029525829490296578
jsd result of udd1: 0.04605415048099792
jsd result of udd2: 0.034677396673652085
jsd result of kdd: 0.04457804118552733
---------Result of circuit G

In [11]:
gs_results_data

{'GS3': [0.03900207408294435,
  0.03182899355288701,
  0.04340882872787646,
  0.06359335351421432,
  0.03202563169941431,
  0.0679296545368999,
  0.05139029924730704,
  0.04432338990842704,
  0.11887832300836947,
  0.03970590428297073,
  0.03105083108013777],
 'GS4': [0.035637905844616076,
  0.04310968907058414,
  0.039363888799197,
  0.04415232832299162,
  0.036547246485533716,
  0.04606141538586049,
  0.059055680890087794,
  0.029525829490296578,
  0.04605415048099792,
  0.034677396673652085,
  0.04457804118552733],
 'GS5': [0.044566498118753455,
  0.04880439092843728,
  0.04621341429979027,
  0.04743459873887378,
  0.04081551164647816,
  0.046646211138714096,
  0.06063297095300158,
  0.05863591008308232,
  0.051851359570726265,
  0.04662051253970806,
  0.056163613414974674],
 'GS6': [0.05533938952045888,
  0.05716642180619514,
  0.05850743071584499,
  0.06349619689295319,
  0.054689392972568154,
  0.045947892740012285,
  0.06974890692062488,
  0.05931816999926025,
  0.05895621328584

In [12]:
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = gs_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names[:-2]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2], linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)
# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for GS circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_GS_toronto_0530.pdf', bbox_inches='tight', pad_inches=0)

In [12]:
qft_job_ids = []
qft_jobs = []

for circuit in qft_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    qft_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    qft_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d7374bb975a1a1efb4d4
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d73a1ea57f7dbfebd62a
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d73d32e176113693c9e9
if applied DD for None: False
if applied DD for N

In [13]:
for job_id in qft_job_ids:
    print(job_id)

6295d7374bb975a1a1efb4d4
6295d73a1ea57f7dbfebd62a
6295d73d32e176113693c9e9
6295d7404bb97546c0efb4d5
6295d7450212ff8e08251f24
6295d74b9baeab0cec4acd7f
6295d75132e1763ac293c9ea
6295d7580212ff6c9d251f25
6295d75e4ad655ff7e9e4aaf
6295d7699baeabd5a44acd80
6295d77532e17612e593c9eb
6295d7814bb9756eb2efb4d7


In [13]:
from scipy.spatial.distance import jensenshannon
qft_jsds = []

qft_job_ids = [
    '6295d7374bb975a1a1efb4d4',
    '6295d73a1ea57f7dbfebd62a',
    '6295d73d32e176113693c9e9',
    '6295d7404bb97546c0efb4d5',
    '6295d7450212ff8e08251f24',
    '6295d74b9baeab0cec4acd7f',
    '6295d75132e1763ac293c9ea',
    '6295d7580212ff6c9d251f25',
    '6295d75e4ad655ff7e9e4aaf',
    '6295d7699baeabd5a44acd80',
    '6295d77532e17612e593c9eb',
    '6295d7814bb9756eb2efb4d7'
]

shots = 8192

for i, circuit in enumerate(qft_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(qft_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    qft_jsds.append(jsd)

circuit_names = ['QFT3', 'QFT4', 'QFT5', 'QFT6', 'QFT7', 'QFT8', 'QFT9', 'QFT10', 'QFT11', 'QFT12', 'QFT13', 'QFT14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

qft_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(qft_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    qft_results_data[name] = qft_jsds[i]

---------Result of circuit QFT3 -------------
jsd result of no_dd: 0.12103083603405687
jsd result of hahn_X: 0.12752976707696195
jsd result of hahn_Y: 0.12093709025937263
jsd result of CP: 0.09808072470996312
jsd result of CPMG: 0.09556159060484883
jsd result of xy4: 0.10238201722252441
jsd result of xy8: 0.10815255021503045
jsd result of xy16: 0.10382296867043776
jsd result of udd1: 0.10245413410457499
jsd result of udd2: 0.09097797533479107
jsd result of kdd: 0.059373568625166434
---------Result of circuit QFT4 -------------
jsd result of no_dd: 0.05729200619640445
jsd result of hahn_X: 0.058551346801456856
jsd result of hahn_Y: 0.05789461823152661
jsd result of CP: 0.06078791138625167
jsd result of CPMG: 0.05078108404754918
jsd result of xy4: 0.07885298002760495
jsd result of xy8: 0.06149482140906805
jsd result of xy16: 0.06643837230867673
jsd result of udd1: 0.09433952617382041
jsd result of udd2: 0.08982189613644245
jsd result of kdd: 0.07468137628977141
---------Result of circuit

In [14]:
qft_results_data

{'QFT3': [0.12103083603405687,
  0.12752976707696195,
  0.12093709025937263,
  0.09808072470996312,
  0.09556159060484883,
  0.10238201722252441,
  0.10815255021503045,
  0.10382296867043776,
  0.10245413410457499,
  0.09097797533479107,
  0.059373568625166434],
 'QFT4': [0.05729200619640445,
  0.058551346801456856,
  0.05789461823152661,
  0.06078791138625167,
  0.05078108404754918,
  0.07885298002760495,
  0.06149482140906805,
  0.06643837230867673,
  0.09433952617382041,
  0.08982189613644245,
  0.07468137628977141],
 'QFT5': [0.15654171611659962,
  0.15295771473720696,
  0.1531894501769698,
  0.10295495894755276,
  0.13798327061357493,
  0.13262374768750776,
  0.1485067883261896,
  0.17606065250782493,
  0.13272190058259126,
  0.1242074030171476,
  0.1285209016439886],
 'QFT6': [0.137639556249265,
  0.1453983077185718,
  0.13817691755713746,
  0.10797451362976321,
  0.08856925850265811,
  0.12239081297970505,
  0.0982280675765275,
  0.10592526159109095,
  0.13976488401129078,
  0.1

In [15]:
# import matplotlib.pyplot as plt
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = qft_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names[:-2]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2], linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)
# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for QFT circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_QFT_toronto_0530.pdf', bbox_inches='tight', pad_inches=0)